# Facial Expression Recognition and Valence/Arousal Prediction

## Overview
This notebook implements a multi-task deep learning approach for:
1. **Facial Expression Classification**: Classifying emotions (Neutral, Happy, Sad, Surprise, Fear, Disgust, Anger, Contempt)
2. **Valence Prediction**: Predicting positive/negative emotional state [-1, +1]  
3. **Arousal Prediction**: Predicting excitement/calmness level [-1, +1]

## Dataset Information
- **Images**: 224x224 RGB facial images
- **Annotations**: Stored in .npy files with expression labels, valence/arousal values, and facial landmarks

## Models
This notebook focuses on two high-performance architectures:
- **ResNet50**: Deep residual network with skip connections
- **EfficientNetB1**: Optimized architecture with compound scaling

Each model uses transfer learning with multi-task learning for simultaneous emotion classification and valence/arousal regression.

## 1. Import Required Libraries

In [2]:
# Core libraries for deep learning and data processing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import glob
import warnings
from pathlib import Path
import random
from tqdm import tqdm

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, ResNet101, 
    EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4,
    MobileNetV2, DenseNet121, InceptionV3, Xception
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Scikit-learn for metrics and preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score, roc_auc_score,
    precision_recall_curve, auc, confusion_matrix, classification_report
)
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Statistical and correlation metrics
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine

# Visualization and plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Configure warnings and display
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# GPU configuration
print("GPU Available: ", tf.config.list_physical_devices('GPU'))
if tf.config.list_physical_devices('GPU'):
    print("Using GPU for training")
    # Configure GPU memory growth
    for gpu in tf.config.experimental.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("Using CPU for training")

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"OpenCV version: {cv2.__version__}")

2025-09-25 19:28:05.345171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758828485.544811      78 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758828485.600485      78 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using GPU for training
TensorFlow version: 2.18.0
Keras version: 3.8.0
NumPy version: 1.26.4
OpenCV version: 4.11.0


In [3]:
# GPU Configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"GPU available: {gpus[0]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found")

is_kaggle = os.path.exists('/kaggle/input')

GPU available: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## 2. Configuration and Constants

In [4]:
# Dataset configuration
DATASET_PATH = "Dataset"
IMAGES_PATH = os.path.join(DATASET_PATH, "images")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations")

# Model configuration
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_CHANNELS = 3
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# Class labels for emotions
EMOTION_LABELS = {
    0: 'Neutral',
    1: 'Happy', 
    2: 'Sad',
    3: 'Surprise',
    4: 'Fear',
    5: 'Disgust',
    6: 'Anger',
    7: 'Contempt'
}

NUM_CLASSES = len(EMOTION_LABELS)

# Training configuration
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001
VALIDATION_SPLIT = 0.2
EARLY_STOPPING_PATIENCE = 8  # Adjusted for 20 epochs
REDUCE_LR_PATIENCE = 4       # Adjusted for 20 epochs

# Model architectures to compare
MODEL_ARCHITECTURES = [
    'VGG16', 'VGG19', 'ResNet50', 'ResNet101',
    'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2',
    'MobileNetV2', 'DenseNet121'
]

# Loss weights for multi-task learning
EMOTION_LOSS_WEIGHT = 1.0
VALENCE_LOSS_WEIGHT = 1.0
AROUSAL_LOSS_WEIGHT = 1.0

print(f"Dataset path: {DATASET_PATH}")
print(f"Images path: {IMAGES_PATH}")
print(f"Annotations path: {ANNOTATIONS_PATH}")
print(f"Input shape: {INPUT_SHAPE}")
print(f"Number of emotion classes: {NUM_CLASSES}")
print(f"Emotion labels: {EMOTION_LABELS}")

Dataset path: Dataset
Images path: Dataset/images
Annotations path: Dataset/annotations
Input shape: (224, 224, 3)
Number of emotion classes: 8
Emotion labels: {0: 'Neutral', 1: 'Happy', 2: 'Sad', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger', 7: 'Contempt'}


In [5]:
# Auto-detect environment and set appropriate paths
is_kaggle = os.path.exists('/kaggle/input')

if is_kaggle:
    print("Kaggle environment detected - applying optimizations")
    
    # Kaggle-optimized training settings
    BATCH_SIZE = 16  # Reduced for GPU memory efficiency
    EPOCHS = 50      # Reduced for time limits
    EARLY_STOPPING_PATIENCE = 10
    
    # Results path
    RESULTS_PATH = "/kaggle/working"
    
else:
    print("Running locally")
    # Keep original local paths
    DATASET_PATH = "Dataset"
    IMAGES_PATH = os.path.join(DATASET_PATH, "images")
    ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations")
    RESULTS_PATH = "results"

# Create results directory
if 'RESULTS_PATH' in locals():
    os.makedirs(RESULTS_PATH, exist_ok=True)
    print(f"Results will be saved to: {RESULTS_PATH}")

# Print final configuration
print(f"\nFinal Configuration:")
print(f"   Dataset path: {DATASET_PATH if 'DATASET_PATH' in locals() else 'Not found'}")
print(f"   Images path: {IMAGES_PATH if IMAGES_PATH else 'Not found'}")
print(f"   Annotations path: {ANNOTATIONS_PATH if ANNOTATIONS_PATH else 'Not found'}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")

print("\nUpdating dataset paths for Kaggle")

# Find the actual dataset structure in Kaggle
dataset_root = "/kaggle/input/affect/Dataset/Dataset"
DATASET_PATH = dataset_root

# Auto-detect image and annotation directories
images_found = False
annotations_found = False

for root, dirs, files in os.walk(dataset_root):
    for dir_name in dirs:
        if "image" in dir_name.lower():
            IMAGES_PATH = os.path.join(root, dir_name)
            print(f"Found images in: {IMAGES_PATH}")
            images_found = True
        elif "annotation" in dir_name.lower():
            ANNOTATIONS_PATH = os.path.join(root, dir_name)
            print(f"Found annotations in: {ANNOTATIONS_PATH}")
            annotations_found = True

# If standard names not found, check the root directory contents
if not images_found or not annotations_found:
    print(f"\nContents of {dataset_root}:")
    contents = os.listdir(dataset_root)
    for item in contents:
        item_path = os.path.join(dataset_root, item)
        if os.path.isdir(item_path):
            item_contents = os.listdir(item_path)[:5]
            print(f"   {item}/ contains: {item_contents}")
            
            # Check if this could be images or annotations
            if any(f.lower().endswith(('.jpg', '.jpeg', '.png')) for f in item_contents):
                IMAGES_PATH = item_path
                print(f"   This appears to be the images directory!")
                images_found = True
            elif any(f.endswith('.npy') for f in item_contents):
                ANNOTATIONS_PATH = item_path
                print(f"   This appears to be the annotations directory!")
                annotations_found = True

print(f"\nFinal paths:")
print(f"   Images: {IMAGES_PATH if images_found else 'Not found'}")
print(f"   Annotations: {ANNOTATIONS_PATH if annotations_found else 'Not found'}")

Kaggle environment detected - applying optimizations
Results will be saved to: /kaggle/working

Final Configuration:
   Dataset path: Dataset
   Images path: Dataset/images
   Annotations path: Dataset/annotations
   Batch size: 16
   Epochs: 50

Updating dataset paths for Kaggle
Found annotations in: /kaggle/input/affect/Dataset/Dataset/annotations
Found images in: /kaggle/input/affect/Dataset/Dataset/images

Final paths:
   Images: /kaggle/input/affect/Dataset/Dataset/images
   Annotations: /kaggle/input/affect/Dataset/Dataset/annotations

Final paths:
   Images: /kaggle/input/affect/Dataset/Dataset/images
   Annotations: /kaggle/input/affect/Dataset/Dataset/annotations


In [6]:
# Update paths based on Kaggle dataset structure
print(" Updating dataset paths for Kaggle")

# Set the correct Kaggle dataset path
DATASET_PATH = "/kaggle/input/affect/Dataset"
IMAGES_PATH = os.path.join(DATASET_PATH, "images")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations")

print(f" Updated Dataset path: {DATASET_PATH}")
print(f"  Updated Images path: {IMAGES_PATH}")
print(f" Updated Annotations path: {ANNOTATIONS_PATH}")

# Verify paths exist
print(f"\n✓ Checking paths:")
print(f"   Dataset exists: {os.path.exists(DATASET_PATH)}")
print(f"   Images exists: {os.path.exists(IMAGES_PATH)}")
print(f"   Annotations exists: {os.path.exists(ANNOTATIONS_PATH)}")

if os.path.exists(IMAGES_PATH):
    image_count = len([f for f in os.listdir(IMAGES_PATH) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"   📸 Image files found: {image_count}")

if os.path.exists(ANNOTATIONS_PATH):
    annotation_count = len([f for f in os.listdir(ANNOTATIONS_PATH) if f.endswith('.npy')])
    print(f"    Annotation files found: {annotation_count}")

 Updating dataset paths for Kaggle
 Updated Dataset path: /kaggle/input/affect/Dataset
  Updated Images path: /kaggle/input/affect/Dataset/images
 Updated Annotations path: /kaggle/input/affect/Dataset/annotations

✓ Checking paths:
   Dataset exists: True
   Images exists: False
   Annotations exists: False


In [7]:
# Set the correct final paths
DATASET_PATH = "/kaggle/input/affect/Dataset/Dataset"
IMAGES_PATH = "/kaggle/input/affect/Dataset/Dataset/images"
ANNOTATIONS_PATH = "/kaggle/input/affect/Dataset/Dataset/annotations"

print("Final dataset configuration:")
print(f"Dataset path: {DATASET_PATH}")
print(f"Images path: {IMAGES_PATH}")
print(f"Annotations path: {ANNOTATIONS_PATH}")

# Verify and count files
image_files = [f for f in os.listdir(IMAGES_PATH) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
annotation_files = [f for f in os.listdir(ANNOTATIONS_PATH) if f.endswith('.npy')]

print(f"\nDataset Statistics:")
print(f"   Total image files: {len(image_files)}")
print(f"   Total annotation files: {len(annotation_files)}")

# Expression files breakdown
exp_files = [f for f in annotation_files if '_exp.npy' in f]
val_files = [f for f in annotation_files if '_val.npy' in f]
aro_files = [f for f in annotation_files if '_aro.npy' in f]
lnd_files = [f for f in annotation_files if '_lnd.npy' in f]

print(f"   Expression files: {len(exp_files)}")
print(f"   Valence files: {len(val_files)}")
print(f"   Arousal files: {len(aro_files)}")
print(f"   Landmark files: {len(lnd_files)}")

# Sample verification
file_ids = [f.split('_')[0] for f in exp_files]
sample_ids = file_ids[:5]

for file_id in sample_ids:
    image_file = f"{file_id}.jpg"
    if os.path.exists(os.path.join(IMAGES_PATH, image_file)):
        # Check annotation files
        exp_file = os.path.join(ANNOTATIONS_PATH, f"{file_id}_exp.npy")
        val_file = os.path.join(ANNOTATIONS_PATH, f"{file_id}_val.npy")
        aro_file = os.path.join(ANNOTATIONS_PATH, f"{file_id}_aro.npy")
        
        if os.path.exists(exp_file) and os.path.exists(val_file) and os.path.exists(aro_file):
            print(f"   {file_id}: Image + Annotations available")

Final dataset configuration:
Dataset path: /kaggle/input/affect/Dataset/Dataset
Images path: /kaggle/input/affect/Dataset/Dataset/images
Annotations path: /kaggle/input/affect/Dataset/Dataset/annotations

Dataset Statistics:
   Total image files: 3999
   Total annotation files: 15996
   Expression files: 3999
   Valence files: 3999
   Arousal files: 3999
   Landmark files: 3999
   2820: Image + Annotations available
   1757: Image + Annotations available
   2691: Image + Annotations available
   406: Image + Annotations available
   2966: Image + Annotations available


## 3. Dataset Loading and Exploration

In [8]:
def explore_dataset():
    """Explore the facial expression dataset structure and contents"""
    print("="*50)
    print("DATASET EXPLORATION")
    print("="*50)
    
    # Check if dataset paths exist
    if not os.path.exists(DATASET_PATH):
        print(f"Dataset path not found: {DATASET_PATH}")
        return
    
    if not os.path.exists(IMAGES_PATH):
        print(f"Images path not found: {IMAGES_PATH}")
        return
        
    if not os.path.exists(ANNOTATIONS_PATH):
        print(f"Annotations path not found: {ANNOTATIONS_PATH}")
        return
    
    # Get list of image files
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(IMAGES_PATH, ext)))
    
    # Get list of annotation files
    annotation_files = glob.glob(os.path.join(ANNOTATIONS_PATH, "*.npy"))
    
    print(f"Dataset path: {DATASET_PATH}")
    print(f"Total image files: {len(image_files)}")
    print(f"Total annotation files: {len(annotation_files)}")
    
    # Analyze annotation types
    exp_files = [f for f in annotation_files if f.endswith('_exp.npy')]
    val_files = [f for f in annotation_files if f.endswith('_val.npy')]
    aro_files = [f for f in annotation_files if f.endswith('_aro.npy')]
    lnd_files = [f for f in annotation_files if f.endswith('_lnd.npy')]
    
    print(f"Expression files: {len(exp_files)}")
    print(f"Valence files: {len(val_files)}")
    print(f"Arousal files: {len(aro_files)}")
    print(f"Landmark files: {len(lnd_files)}")
    
    # Extract unique file IDs
    file_ids = set()
    for f in annotation_files:
        filename = os.path.basename(f)
        file_id = filename.split('_')[0]
        file_ids.add(file_id)
    
    print(f"Unique file IDs: {len(file_ids)}")
    
    return sorted(list(file_ids))

# Explore the dataset
file_ids = explore_dataset()

DATASET EXPLORATION
Dataset path: /kaggle/input/affect/Dataset/Dataset
Total image files: 3999
Total annotation files: 15996
Expression files: 3999
Valence files: 3999
Arousal files: 3999
Landmark files: 3999
Unique file IDs: 3999


## 4. Custom Evaluation Metrics Implementation

In [9]:
def krippendorff_alpha(data, level_of_measurement='interval'):
    """
    Calculate Krippendorf's Alpha for inter-rater reliability
    
    Args:
        data: Array of shape (n_raters, n_items) or (n_items, n_raters)
        level_of_measurement: 'nominal', 'ordinal', 'interval', or 'ratio'
    
    Returns:
        float: Krippendorf's Alpha value
    """
    # Simplified implementation for interval data
    # For complete implementation, use krippendorff library
    from sklearn.metrics import cohen_kappa_score
    
    if data.shape[0] == 2:  # Two raters
        return cohen_kappa_score(data[0], data[1])
    else:
        # For multiple raters, use average pairwise kappa
        kappas = []
        n_raters = data.shape[0]
        for i in range(n_raters):
            for j in range(i+1, n_raters):
                kappa = cohen_kappa_score(data[i], data[j])
                kappas.append(kappa)
        return np.mean(kappas) if kappas else 0.0

def sign_agreement_metric(y_true, y_pred):
    """
    Calculate Sign Agreement Metric (SAGR)
    Penalizes incorrect sign alongside deviation from value
    
    Args:
        y_true: Ground truth values
        y_pred: Predicted values
    
    Returns:
        float: SAGR score
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate sign agreement
    sign_true = np.sign(y_true)
    sign_pred = np.sign(y_pred)
    sign_agreement = np.mean(sign_true == sign_pred)
    
    # Calculate magnitude correlation
    correlation, _ = pearsonr(np.abs(y_true), np.abs(y_pred))
    
    # Combine sign agreement and correlation
    sagr = sign_agreement * correlation if not np.isnan(correlation) else sign_agreement
    
    return sagr

def concordance_correlation_coefficient(y_true, y_pred):
    """
    Calculate Concordance Correlation Coefficient (CCC)
    Combines Pearson correlation with accuracy
    
    Args:
        y_true: Ground truth values
        y_pred: Predicted values
    
    Returns:
        float: CCC value
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Pearson correlation coefficient
    correlation, _ = pearsonr(y_true, y_pred)
    
    # Means and variances
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    
    # Standard deviations
    sd_true = np.sqrt(var_true)
    sd_pred = np.sqrt(var_pred)
    
    # Concordance correlation coefficient
    numerator = 2 * correlation * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred) ** 2
    
    ccc = numerator / denominator if denominator != 0 else 0
    
    return ccc

def evaluate_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """
    Calculate comprehensive classification metrics
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (optional)
    
    Returns:
        dict: Dictionary of metric values
    """
    metrics = {}
    
    # Basic metrics
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['f1_score'] = f1_score(y_true, y_pred, average='weighted')
    metrics['cohen_kappa'] = cohen_kappa_score(y_true, y_pred)
    
    # Krippendorf's Alpha (using Cohen's Kappa as approximation)
    metrics['krippendorff_alpha'] = metrics['cohen_kappa']
    
    # AUC metrics (if probabilities provided)
    if y_pred_proba is not None:
        try:
            metrics['auc_roc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='macro')
        except:
            metrics['auc_roc'] = 0.0
        
        try:
            # Calculate AUC-PR for each class and average
            auc_pr_scores = []
            for i in range(y_pred_proba.shape[1]):
                y_true_binary = (y_true == i).astype(int)
                precision, recall, _ = precision_recall_curve(y_true_binary, y_pred_proba[:, i])
                auc_pr = auc(recall, precision)
                auc_pr_scores.append(auc_pr)
            metrics['auc_pr'] = np.mean(auc_pr_scores)
        except:
            metrics['auc_pr'] = 0.0
    
    return metrics

def evaluate_regression_metrics(y_true, y_pred):
    """
    Calculate comprehensive regression metrics
    
    Args:
        y_true: True values
        y_pred: Predicted values
    
    Returns:
        dict: Dictionary of metric values
    """
    metrics = {}
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Basic metrics
    metrics['rmse'] = np.sqrt(np.mean((y_true - y_pred) ** 2))
    metrics['mae'] = np.mean(np.abs(y_true - y_pred))
    metrics['r2'] = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
    
    # Correlation
    correlation, p_value = pearsonr(y_true, y_pred)
    metrics['correlation'] = correlation if not np.isnan(correlation) else 0.0
    metrics['correlation_p_value'] = p_value if not np.isnan(p_value) else 1.0
    
    # Custom metrics
    metrics['sagr'] = sign_agreement_metric(y_true, y_pred)
    metrics['ccc'] = concordance_correlation_coefficient(y_true, y_pred)
    
    return metrics

print("✅ Custom evaluation metrics implemented successfully!")
print(" Available classification metrics: accuracy, f1_score, cohen_kappa, krippendorff_alpha, auc_roc, auc_pr")
print(" Available regression metrics: rmse, mae, r2, correlation, sagr, ccc")

✅ Custom evaluation metrics implemented successfully!
 Available classification metrics: accuracy, f1_score, cohen_kappa, krippendorff_alpha, auc_roc, auc_pr
 Available regression metrics: rmse, mae, r2, correlation, sagr, ccc


## 5. Multi-Task CNN Model Architecture

In [10]:
def create_multitask_model(base_model_name='EfficientNetB0', input_shape=INPUT_SHAPE, 
                          num_classes=NUM_CLASSES, trainable_layers=50):
    """
    Create a multi-task CNN model for emotion classification and valence/arousal regression
    
    Args:
        base_model_name (str): Name of the base CNN architecture
        input_shape (tuple): Input shape for images
        num_classes (int): Number of emotion classes
        trainable_layers (int): Number of top layers to make trainable
    
    Returns:
        tensorflow.keras.Model: Compiled multi-task model
    """
    
    # Input layer
    inputs = layers.Input(shape=input_shape, name='image_input')
    
    # Get base model - only ResNet50 and EfficientNetB1 supported
    base_models = {
        'ResNet50': ResNet50,
        'EfficientNetB1': EfficientNetB1
    }
    
    if base_model_name not in base_models:
        raise ValueError(f"Unsupported base model: {base_model_name}")
    
    # Create base model with ImageNet weights
    base_model = base_models[base_model_name](
        weights='imagenet',
        include_top=False,
        input_tensor=inputs
    )
    
    # Freeze initial layers
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False
    
    # Add global average pooling
    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(base_model.output)
    
    # Shared feature layer
    x = layers.Dense(512, activation='relu', name='shared_features')(x)
    x = layers.Dropout(0.5, name='shared_dropout')(x)
    
    # Emotion classification head
    emotion_features = layers.Dense(256, activation='relu', name='emotion_features')(x)
    emotion_features = layers.Dropout(0.3, name='emotion_dropout')(emotion_features)
    emotion_output = layers.Dense(num_classes, activation='softmax', name='emotion_output')(emotion_features)
    
    # Valence regression head
    valence_features = layers.Dense(128, activation='relu', name='valence_features')(x)
    valence_features = layers.Dropout(0.3, name='valence_dropout')(valence_features)
    valence_output = layers.Dense(1, activation='tanh', name='valence_output')(valence_features)
    
    # Arousal regression head
    arousal_features = layers.Dense(128, activation='relu', name='arousal_features')(x)
    arousal_features = layers.Dropout(0.3, name='arousal_dropout')(arousal_features)
    arousal_output = layers.Dense(1, activation='tanh', name='arousal_output')(arousal_features)
    
    # Create model
    model = models.Model(
        inputs=inputs,
        outputs=[emotion_output, valence_output, arousal_output],
        name=f'MultiTask_{base_model_name}'
    )
    
    return model

def compile_multitask_model(model, learning_rate=LEARNING_RATE):
    """
    Compile multi-task model with appropriate loss functions and metrics
    
    Args:
        model: Keras model to compile
        learning_rate: Learning rate for optimizer
    
    Returns:
        Compiled model
    """
    
    # Define losses
    losses = {
        'emotion_output': 'categorical_crossentropy',
        'valence_output': 'mse',
        'arousal_output': 'mse'
    }
    
    # Define loss weights
    loss_weights = {
        'emotion_output': EMOTION_LOSS_WEIGHT,
        'valence_output': VALENCE_LOSS_WEIGHT,
        'arousal_output': AROUSAL_LOSS_WEIGHT
    }
    
    # Define metrics
    metrics = {
        'emotion_output': ['accuracy'],
        'valence_output': ['mae'],
        'arousal_output': ['mae']
    }
    
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss=losses,
        loss_weights=loss_weights,
        metrics=metrics
    )
    
    return model

def get_model_summary(model):
    """
    Get detailed model summary including parameter count
    
    Args:
        model: Keras model
    
    Returns:
        dict: Model statistics
    """
    
    # Count parameters using model.count_params() method
    try:
        total_params = model.count_params()
        
        # Count trainable parameters manually
        trainable_params = 0
        for layer in model.layers:
            if layer.trainable:
                layer_params = sum([np.prod(w.shape) for w in layer.get_weights()])
                trainable_params += layer_params
        
        non_trainable_params = total_params - trainable_params
        
    except:
        # Fallback method
        trainable_params = sum([np.prod(w.shape) for w in model.trainable_weights])
        non_trainable_params = sum([np.prod(w.shape) for w in model.non_trainable_weights])
        total_params = trainable_params + non_trainable_params
    
    stats = {
        'total_parameters': int(total_params),
        'trainable_parameters': int(trainable_params),
        'non_trainable_parameters': int(non_trainable_params),
        'model_size_mb': total_params * 4 / (1024 * 1024),  # Assuming float32
        'layers': len(model.layers)
    }
    
    return stats

print("✅ Multi-task model architecture functions created successfully!")
print("  Available architectures:", MODEL_ARCHITECTURES)
print(" Model outputs: emotion_classification, valence_regression, arousal_regression")

✅ Multi-task model architecture functions created successfully!
  Available architectures: ['VGG16', 'VGG19', 'ResNet50', 'ResNet101', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2', 'MobileNetV2', 'DenseNet121']
 Model outputs: emotion_classification, valence_regression, arousal_regression


## 6. Training Pipeline and Model Comparison Framework

This section contains the complete training pipeline and framework for comparing multiple CNN architectures. Run the cells below to:

1. **Load and preprocess the dataset**
2. **Create train/validation splits**
3. **Train multiple CNN models**
4. **Evaluate and compare results**
5. **Generate comprehensive performance reports**

### Next Steps:
1. **Run the data loading cell** to load your dataset
2. **Execute the training loop** to train all models
3. **Analyze results** using the evaluation metrics
4. **Compare architectures** to find the best performing model

### Expected Training Time:
- **CPU**: 8-12 hours per model
- **GPU**: 2-4 hours per model
- **Total**: 16-40 hours for all models (depending on hardware)

### Memory Requirements:
- **Minimum**: 8GB RAM
- **Recommended**: 16GB RAM + 6GB GPU memory
- **Batch size**: Adjust based on available memory

## 7. Data Loading and Preprocessing Pipeline

In [11]:
# DATASET LOADING AND PREPROCESSING FOR 20 EPOCHS TRAINING
def load_annotations(annotations_path, file_id):
    """Load annotations for a given file ID"""
    try:
        annotations = {}
        
        # Load expression (emotion class)
        exp_path = os.path.join(annotations_path, f"{file_id}_exp.npy")
        if os.path.exists(exp_path):
            annotations['expression'] = np.load(exp_path).item()
        
        # Load valence
        val_path = os.path.join(annotations_path, f"{file_id}_val.npy") 
        if os.path.exists(val_path):
            annotations['valence'] = np.load(val_path).item()
            
        # Load arousal
        aro_path = os.path.join(annotations_path, f"{file_id}_aro.npy")
        if os.path.exists(aro_path):
            annotations['arousal'] = np.load(aro_path).item()
            
        return annotations
        
    except Exception as e:
        return {}

def load_and_preprocess_dataset_fast(images_path, annotations_path, file_ids, sample_size=1000):
    """Fast dataset loading optimized for training"""
    
    print(f"Loading {sample_size} samples for training...")
    print(f"Images path: {images_path}")
    print(f"Annotations path: {annotations_path}")
    print(f"Available file IDs: {len(file_ids)}")
    
    # Sample file IDs
    sample_file_ids = np.random.choice(file_ids, min(sample_size, len(file_ids)), replace=False)
    print(f"Randomly selected {len(sample_file_ids)} samples to load")
    
    images = []
    emotions = []
    valences = []
    arousals = []
    valid_ids = []
    
    # Counters for status tracking
    successful_loads = 0
    image_errors = 0
    annotation_errors = 0
    missing_files = 0
    
    print(f"\nStarting data loading...")
    
    for i, file_id in enumerate(tqdm(sample_file_ids, desc="Loading data")):
        try:
            # Load image
            image_path = os.path.join(images_path, f"{file_id}.jpg")
            if os.path.exists(image_path):
                image = cv2.imread(image_path)
                if image is not None:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
                    image = image.astype(np.float32) / 255.0
                    
                    # Load annotations
                    annotations = load_annotations(annotations_path, file_id)
                    
                    if 'expression' in annotations and 'valence' in annotations and 'arousal' in annotations:
                        images.append(image)
                        emotions.append(int(annotations['expression']))
                        valences.append(float(annotations['valence']))
                        arousals.append(float(annotations['arousal']))
                        valid_ids.append(file_id)
                        successful_loads += 1
                        
                        # Show progress every 100 samples
                        if (i + 1) % 100 == 0:
                            print(f"Progress: {successful_loads}/{i+1} samples loaded successfully ({successful_loads/(i+1)*100:.1f}%)")
                    else:
                        annotation_errors += 1
                        if annotation_errors <= 5:  # Only show first 5 annotation errors
                            print(f"File {file_id}: Missing annotations (exp: {'expression' in annotations}, val: {'valence' in annotations}, aro: {'arousal' in annotations})")
                else:
                    image_errors += 1
                    if image_errors <= 5:  # Only show first 5 image errors
                        print(f"File {file_id}: Failed to read image")
            else:
                missing_files += 1
                if missing_files <= 5:  # Only show first 5 missing files
                    print(f"File {file_id}: Image file not found")
                        
        except Exception as e:
            if i < 10:  # Only show first 10 general errors
                print(f"Error processing {file_id}: {str(e)}")
            continue
    
    # Convert to numpy arrays
    X = np.array(images)
    y_emotions = np.array(emotions)
    y_valence = np.array(valences)
    y_arousal = np.array(arousals)
    
    # Final status report
    print(f"\nLOADING COMPLETE!")
    print(f"   Successfully loaded: {successful_loads} samples")
    print(f"   Image errors: {image_errors}")
    print(f"   Annotation errors: {annotation_errors}")
    print(f"   Missing files: {missing_files}")
    print(f"   Success rate: {successful_loads/len(sample_file_ids)*100:.1f}%")
    
    if len(X) > 0:
        print(f"\nDATASET STATISTICS:")
        print(f"   Image shape: {X.shape}")
        print(f"   Emotion range: [{y_emotions.min()}, {y_emotions.max()}]")
        print(f"   Valence range: [{y_valence.min():.3f}, {y_valence.max():.3f}]")
        print(f"   Arousal range: [{y_arousal.min():.3f}, {y_arousal.max():.3f}]")
        print(f"   Ready for training!")
    else:
        print(f"\nLOADING FAILED: No valid samples loaded!")
        print(f"   Check paths and file availability")
    
    return X, y_emotions, y_valence, y_arousal, valid_ids

# Get file IDs from existing exploration
file_ids = sorted([f.split('_')[0] for f in os.listdir(ANNOTATIONS_PATH) if f.endswith('_exp.npy')])

print(f"Available file IDs: {len(file_ids)}")
print(f"Ready for enhanced dataset loading with detailed status!")

# Load dataset optimized for training
SAMPLE_SIZE = 1000  # Balanced sample size for 20 epoch training

X, y_emotions, y_valence, y_arousal, valid_ids = load_and_preprocess_dataset_fast(
    IMAGES_PATH, ANNOTATIONS_PATH, file_ids, sample_size=SAMPLE_SIZE
)

Available file IDs: 3999
Ready for enhanced dataset loading with detailed status!
Loading 1000 samples for training...
Images path: /kaggle/input/affect/Dataset/Dataset/images
Annotations path: /kaggle/input/affect/Dataset/Dataset/annotations
Available file IDs: 3999
Randomly selected 1000 samples to load

Starting data loading...


Loading data:  11%|█         | 110/1000 [00:03<00:13, 67.65it/s]

Progress: 100/100 samples loaded successfully (100.0%)


Loading data:  21%|██▏       | 213/1000 [00:04<00:08, 90.36it/s]

Progress: 200/200 samples loaded successfully (100.0%)


...

Progress: 800/800 samples loaded successfully (100.0%)


Loading data:  92%|█████████▏| 915/1000 [00:12<00:01, 84.78it/s]

Progress: 900/900 samples loaded successfully (100.0%)


Loading data: 100%|██████████| 1000/1000 [00:13<00:00, 71.59it/s]



Progress: 1000/1000 samples loaded successfully (100.0%)

LOADING COMPLETE!
   Successfully loaded: 1000 samples
   Image errors: 0
   Annotation errors: 0
   Missing files: 0
   Success rate: 100.0%

DATASET STATISTICS:
   Image shape: (1000, 224, 224, 3)
   Emotion range: [0, 7]
   Valence range: [-0.962, 0.982]
   Arousal range: [-0.627, 0.968]
   Ready for training!


## 8. Training Pipeline and Model Comparison

## 9. Model Evaluation and Results Analysis

##  Complete Training Pipeline for All Models

**Ready to train all CNN architectures!** 

The cell below contains the complete training setup for all models. **Uncomment and run it** when you're ready for full training:

- **Dataset**: All 3,999 samples
- **Models**: All 9 CNN architectures  
- **Training**: 50 epochs with early stopping
- **Evaluation**: Complete metrics comparison

**⚠️ Expected Training Time: 15-25 hours on Kaggle GPUs**

## Efficient Model Screening Strategy

To optimize training time and resource usage, we'll implement a two-phase approach:

**Phase 1: Quick Screening (5 epochs each)**
- Test 6 best CNN models for facial expression recognition
- Identify top 3 performers based on validation metrics
- Estimated time: 2-3 hours

**Phase 2: Full Training (25-30 epochs)**
- Train only the top 3 models with full epochs
- Comprehensive evaluation and comparison
- Estimated time: 3-4 hours

**Total estimated time: 5-7 hours (vs 15-25 hours for all models)**

## Phase 2: Full Training of Top Models

Now we'll train the top 3 performing models with full epochs for comprehensive evaluation.

## Final Results Summary and Comparison

Comprehensive comparison of the top performing models with detailed metrics and recommendations.

## Full Training Pipeline - All Models (20 Epochs)

With Tesla P100 confirmed, we'll now train all 9 CNN architectures with optimized settings:
- **20 epochs** with early stopping
- **Fine-tuned hyperparameters** for P100
- **Advanced callbacks** for optimal training
- **Memory optimization** for 16GB GPU
- **Comprehensive evaluation** for each model

In [12]:
# MODEL CONFIGURATION
MODEL_ARCHITECTURES = ['ResNet50', 'EfficientNetB1']

# COMPREHENSIVE MODEL EVALUATION FUNCTION
def evaluate_model_comprehensive(model, X_test, y_test_dict, model_name):
    """
    Comprehensive evaluation of multi-task model
    """
    print(f"Evaluating {model_name}...")
    
    # Make predictions
    predictions = model.predict(X_test, batch_size=32, verbose=0)
    
    # Extract predictions for each task
    y_emotion_pred = predictions[0]  # Emotion classification logits
    y_valence_pred = predictions[1].flatten()  # Valence regression
    y_arousal_pred = predictions[2].flatten()  # Arousal regression
    
    # Ground truth
    y_emotion_true = y_test_dict['emotion_output']
    y_valence_true = y_test_dict['valence_output']
    y_arousal_true = y_test_dict['arousal_output']
    
    # Convert emotion predictions to class predictions
    y_emotion_pred_classes = np.argmax(y_emotion_pred, axis=1)
    y_emotion_true_classes = np.argmax(y_emotion_true, axis=1)
    
    # Evaluate emotion classification
    emotion_metrics = evaluate_classification_metrics(
        y_emotion_true_classes, y_emotion_pred_classes, y_emotion_pred
    )
    
    # Evaluate valence regression
    valence_metrics = evaluate_regression_metrics(y_valence_true, y_valence_pred)
    
    # Evaluate arousal regression  
    arousal_metrics = evaluate_regression_metrics(y_arousal_true, y_arousal_pred)
    
    results = {
        'emotion_metrics': emotion_metrics,
        'valence_metrics': valence_metrics,
        'arousal_metrics': arousal_metrics
    }
    
    print(f"  Emotion Accuracy: {emotion_metrics['accuracy']:.4f}")
    print(f"  Valence RMSE: {valence_metrics['rmse']:.4f}")
    print(f"  Arousal RMSE: {arousal_metrics['rmse']:.4f}")
    
    return results

print("Model configuration updated: ResNet50 and EfficientNetB1 only")
print("Evaluation function ready")

Model configuration updated: ResNet50 and EfficientNetB1 only
Evaluation function ready


In [13]:
# CLEAN TRAINING PIPELINE - RESNET50 AND EFFICIENTNETB1 ONLY
import time

def train_model_clean(model_name):
    """
    Clean training function for a single model
    """
    print(f"Training {model_name}...")
    print("-" * 50)
    
    start_time = time.time()
    
    try:
        # Create model
        model = create_multitask_model(model_name)
        model = compile_multitask_model(model)
        
        # Training callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=EARLY_STOPPING_PATIENCE,
                restore_best_weights=True,
                verbose=1
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=REDUCE_LR_PATIENCE,
                min_lr=1e-7,
                verbose=1
            )
        ]
        
        # Train model
        history = model.fit(
            X_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )
        
        # Evaluate model
        results = evaluate_model_comprehensive(model, X_val, y_val, model_name)
        
        training_time = time.time() - start_time
        results['training_time'] = training_time
        results['epochs_completed'] = len(history.history['loss'])
        
        print(f"Training completed in {training_time/60:.1f} minutes")
        print(f"Epochs completed: {results['epochs_completed']}")
        
        return model, results, history
        
    except Exception as e:
        print(f"Error training {model_name}: {str(e)}")
        return None, {'error': str(e)}, None

def train_both_models():
    """
    Train both ResNet50 and EfficientNetB1 models
    """
    print("="*60)
    print("TRAINING RESNET50 AND EFFICIENTNETB1")
    print("="*60)
    
    # Check data availability
    if not all(var in globals() for var in ['X_train', 'X_val', 'y_train', 'y_val']):
        print("Error: Training data not available. Please run data loading first.")
        return None, None
    
    print(f"Training samples: {X_train.shape[0]}")
    print(f"Validation samples: {X_val.shape[0]}")
    print(f"Epochs per model: {EPOCHS}")
    
    results = {}
    models = {}
    histories = {}
    
    # Train both models
    for model_name in MODEL_ARCHITECTURES:
        print(f"\nTraining {model_name}...")
        model, result, history = train_model_clean(model_name)
        
        if model is not None:
            models[model_name] = model
        results[model_name] = result
        if history is not None:
            histories[model_name] = history
    
    return models, results, histories

print("Clean training pipeline ready")
print(f"Models to train: {MODEL_ARCHITECTURES}")

Clean training pipeline ready
Models to train: ['ResNet50', 'EfficientNetB1']


In [14]:
# RESULTS ANALYSIS FUNCTION
def analyze_results(results):
    """
    Simple results analysis without emojis
    """
    print("="*60)
    print("TRAINING RESULTS ANALYSIS")
    print("="*60)
    
    successful_models = []
    failed_models = []
    
    for model_name, result in results.items():
        if 'error' in result:
            failed_models.append(model_name)
        else:
            successful_models.append(model_name)
    
    print(f"Successful models: {len(successful_models)}/2")
    print(f"Failed models: {len(failed_models)}/2")
    
    if successful_models:
        print("\nPERFORMACE COMPARISON:")
        print("-" * 40)
        
        for model_name in successful_models:
            result = results[model_name]
            print(f"\n{model_name}:")
            print(f"  Emotion Accuracy: {result['emotion_metrics']['accuracy']:.4f}")
            print(f"  Valence RMSE: {result['valence_metrics']['rmse']:.4f}")
            print(f"  Arousal RMSE: {result['arousal_metrics']['rmse']:.4f}")
            print(f"  Training Time: {result['training_time']/60:.1f} minutes")
            print(f"  Epochs: {result['epochs_completed']}")
        
        # Find best model
        best_emotion = max(successful_models, 
                          key=lambda x: results[x]['emotion_metrics']['accuracy'])
        best_valence = min(successful_models, 
                          key=lambda x: results[x]['valence_metrics']['rmse'])
        best_arousal = min(successful_models, 
                          key=lambda x: results[x]['arousal_metrics']['rmse'])
        
        print("\nBEST PERFORMERS:")
        print("-" * 20)
        print(f"Best Emotion: {best_emotion}")
        print(f"Best Valence: {best_valence}")  
        print(f"Best Arousal: {best_arousal}")
        
        # Overall recommendation
        emotion_scores = {name: results[name]['emotion_metrics']['accuracy'] 
                         for name in successful_models}
        overall_best = max(emotion_scores.keys(), key=lambda x: emotion_scores[x])
        
        print(f"\nRECOMMENDED MODEL: {overall_best}")
        print(f"Emotion Accuracy: {emotion_scores[overall_best]:.4f}")
    
    if failed_models:
        print(f"\nFailed models: {failed_models}")
        for model_name in failed_models:
            print(f"  {model_name}: {results[model_name]['error']}")
    
    print("="*60)

print("Results analysis function ready")

Results analysis function ready


# Training Instructions

## Quick Start
1. Run all cells in order to set up the environment and load data
2. Use `train_both_models()` to train both ResNet50 and EfficientNetB1
3. Use `analyze_results(results)` to view performance comparison

## Example Usage
```python
# Train both models
trained_models, results, histories = train_both_models()

# Analyze results
analyze_results(results)
```

The notebook now focuses on the two best-performing architectures with clean, professional code.

In [15]:
# Execute the training pipeline
print("Starting training for ResNet50 and EfficientNetB1...")
print("="*60)

# Check if data is loaded
if 'X' in globals():
    print(f"Dataset loaded: {X.shape}")
    print(f"Emotions: {y_emotions.shape}")
    print(f"Valence: {y_valence.shape}")
    print(f"Arousal: {y_arousal.shape}")
    
    # Split the data into training and validation sets
    from sklearn.model_selection import train_test_split
    
    # Split the data
    X_train, X_val, y_emotions_train, y_emotions_val = train_test_split(
        X, y_emotions, test_size=VALIDATION_SPLIT, random_state=42, stratify=y_emotions
    )
    
    _, _, y_valence_train, y_valence_val = train_test_split(
        X, y_valence, test_size=VALIDATION_SPLIT, random_state=42, stratify=y_emotions
    )
    
    _, _, y_arousal_train, y_arousal_val = train_test_split(
        X, y_arousal, test_size=VALIDATION_SPLIT, random_state=42, stratify=y_emotions
    )
    
    # Convert emotion labels to categorical (one-hot encoded)
    from tensorflow.keras.utils import to_categorical
    y_emotions_train_cat = to_categorical(y_emotions_train, NUM_CLASSES)
    y_emotions_val_cat = to_categorical(y_emotions_val, NUM_CLASSES)
    
    # Prepare train and validation dictionaries with correct output names
    y_train = {
        'emotion_output': y_emotions_train_cat,
        'valence_output': y_valence_train,
        'arousal_output': y_arousal_train
    }
    
    y_val = {
        'emotion_output': y_emotions_val_cat,
        'valence_output': y_valence_val,
        'arousal_output': y_arousal_val
    }
    
    print(f"Training samples: {X_train.shape[0]}")
    print(f"Validation samples: {X_val.shape[0]}")
    print(f"Emotion classes: {y_emotions_train_cat.shape[1]}")
    
    # Train both models
    models, results, histories = train_both_models()
    
    # Analyze results
    if results:
        analyze_results(results)
    else:
        print("Training failed - no results to analyze")
        
else:
    print("Error: Dataset not loaded. Please run the data loading cell first.")

Starting training for ResNet50 and EfficientNetB1...
Dataset loaded: (1000, 224, 224, 3)
Emotions: (1000,)
Valence: (1000,)
Arousal: (1000,)
Training samples: 800
Validation samples: 200
Emotion classes: 8
TRAINING RESNET50 AND EFFICIENTNETB1
Training samples: 800
Validation samples: 200
Epochs per model: 50

Training ResNet50...
Training ResNet50...
--------------------------------------------------
Training samples: 800
Validation samples: 200
Emotion classes: 8
TRAINING RESNET50 AND EFFICIENTNETB1
Training samples: 800
Validation samples: 200
Epochs per model: 50

Training ResNet50...
Training ResNet50...
--------------------------------------------------


I0000 00:00:1758828603.271718      78 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/50
Epoch 1/50


I0000 00:00:1758828634.244551     110 service.cc:148] XLA service 0x796f3c003960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758828634.245365     110 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1758828636.699408     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1758828636.699408     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


 3/50 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - arousal_output_loss: 0.4795 - arousal_output_mae: 0.5654 - emotion_output_accuracy: 0.0660 - emotion_output_loss: 2.4028 - loss: 3.3967 - valence_output_loss: 0.5145 - valence_output_mae: 0.5949

I0000 00:00:1758828646.734248     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 48s 240ms/step - arousal_output_loss: 0.3118 - arousal_output_mae: 0.4456 - emotion_output_accuracy: 0.1128 - emotion_output_loss: 2.2852 - loss: 2.9940 - valence_output_loss: 0.3971 - valence_output_mae: 0.5051 - val_arousal_output_loss: 0.5468 - val_arousal_output_mae: 0.6408 - val_emotion_output_accuracy: 0.1250 - val_emotion_output_loss: 2.7350 - val_loss: 4.1191 - val_valence_output_loss: 0.8169 - val_valence_output_mae: 0.7861 - learning_rate: 0.0010
50/50 ━━━━━━━━━━━━━━━━━━━━ 48s 240ms/step - arousal_output_loss: 0.3118 - arousal_output_mae: 0.4456 - emotion_output_accuracy: 0.1128 - emotion_output_loss: 2.2852 - loss: 2.9940 - valence_output_loss: 0.3971 - valence_output_mae: 0.5051 - val_arousal_output_loss: 0.5468 - val_arousal_output_mae: 0.6408 - val_emotion_output_accuracy: 0.1250 - val_emotion_output_loss: 2.7350 - val_loss: 4.1191 - val_valence_output_loss: 0.8169 - val_valence_output_mae: 0.7861 - learning_rate: 0.0010
Epoch 2/50
Epoch 2/50
50

E0000 00:00:1758828792.070846     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1758828792.279319     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1758828792.279319     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1758828792.598695     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1758828792.598695     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 359ms/step - arousal_output_loss: 0.2200 - arousal_output_mae: 0.3897 - emotion_output_accuracy: 0.1127 - emotion_output_loss: 2.1327 - loss: 2.7785 - valence_output_loss: 0.4258 - valence_output_mae: 0.5304 - val_arousal_output_loss: 0.1514 - val_arousal_output_mae: 0.3445 - val_emotion_output_accuracy: 0.1350 - val_emotion_output_loss: 2.0830 - val_loss: 2.4708 - val_valence_output_loss: 0.2282 - val_valence_output_mae: 0.3824 - learning_rate: 0.0010
50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 359ms/step - arousal_output_loss: 0.2200 - arousal_output_mae: 0.3897 - emotion_output_accuracy: 0.1127 - emotion_output_loss: 2.1327 - loss: 2.7785 - valence_output_loss: 0.4258 - valence_output_mae: 0.5304 - val_arousal_output_loss: 0.1514 - val_arousal_output_mae: 0.3445 - val_emotion_output_accuracy: 0.1350 - val_emotion_output_loss: 2.0830 - val_loss: 2.4708 - val_valence_output_loss: 0.2282 - val_valence_output_mae: 0.3824 - learning_rate: 0.0010
Epoch 2/50
Epoch 2/50
50